# Ejercicio OpenStreetMap
Para este ejercicio vas a trabar con Overpass, que es la API que utiliza los datos de OpenStreetMap. Tras obtener datos geográficos mediante la API, tendrás que representarlos con Plotly.

Hay que obtener los siguientes datos de cada ubicación conseguida en OpenStreetMap
* tipo de amenity
* latitud
* longitud
* nombre
* direccion

**NOTA**: En este Notebook utilizarás gráficas de Plotly, y JupyterLab de momento no lo soporta, a no ser que instales una extensión. Para solucionar esto, abre el Notebook desde Jupyter normal:

![imagen](./img/jupyter.png)

Tienes en [este enlace](https://wiki.openstreetmap.org/wiki/Key:amenity#Healthcare) la lista de *amenities* que vas a usar en el notebook.

## Instalación
Si no tienes instalado Plotly, abre un terminal y escribe:
```
pip install plotly
```

Importa las librerias que vamos a usar en el Notebook

In [1]:
import requests
import json
import pandas as pd

Ataca a la API de OpenStreetmap con el siguiente código de ejemplo

In [2]:
from matplotlib.font_manager import json_dump
import json

overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["amenity"="ice_cream"]
  (40.420801, -3.694702,40.423754, -3.688167); 
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()


<Response [200]>


In [3]:
data

{'version': 0.6,
 'generator': 'Overpass API 0.7.59 e21c39fe',
 'osm3s': {'timestamp_osm_base': '2022-10-31T08:50:39Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': []}

Con el siguiente código podrás adaptar el JSON que devuelve la API, a un formato DataFrame más amigable

In [4]:
def json_to_df(data):
    elements = data['elements']
    places = {'tipo': [], 'lat': [], 'lon': [], 'name': [], 'address': []}
    
    for i in elements:
        
        tipo = i.get('tags', None).get('amenity', None)
        latitude = i.get('lat', None)
        longitude = i.get('lon', None)
        name = i.get('tags', {}).get('name', "NO NAME")
        street = i.get('tags', {}).get('addr:street', "NO STREET")
        number = i.get('tags', {}).get('addr:housenumber', 9999)

        places['tipo'].append(tipo)
        places['lat'].append(latitude)
        places['lon'].append(longitude)
        places['name'].append(name)
        places['address'].append(street + ' ' + str(number))

            
    return pd.DataFrame(places)

Utilizarás esta función para representar los datos en un mapa de Matplotlib

In [5]:
import plotly.express as px

mapbox_access_token = "pk.eyJ1IjoiZGFvcnRpIiwiYSI6ImNrZnF6M3FlczA3cDEyem16YTNzZmV4M2EifQ.846iF0sMSAXv0kwkwUTYjg"

def pinta_mapa(df):
    fig = px.scatter_mapbox(df,
                            lat="lat",
                            lon="lon",
                            color = "tipo",
                      color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=11)
    fig.update_layout(
        mapbox=dict(
            accesstoken=mapbox_access_token
        )
    )

    fig.show()

Como ejemplo, si quiero ver cuántos restaurantes tengo cerca de The Bridge:

In [6]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["amenity"="restaurant"]
  (40.420801, -3.694702,40.423754, -3.688167); 
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()
df = json_to_df(data)
df

<Response [200]>


,tipo,lat,lon,name,address
0,restaurant,40.423086,-3.693547,La Castafiore,NO STREET 9999
1,restaurant,40.422607,-3.693256,Sake Dining Himawari,NO STREET 9999
2,restaurant,40.423315,-3.692506,Ainhoa,Calle de Doña Bárbara de Braganza 12
3,restaurant,40.421960,-3.694270,La Kitchen,NO STREET 9999
4,restaurant,40.423695,-3.691651,Café Espejo,NO STREET 9999
5,restaurant,40.421951,-3.693535,Tampu,Calle de Prim 13
6,restaurant,40.422266,-3.694097,Casa Gades,Calle del Conde de Xiquena 4
7,restaurant,40.423151,-3.694062,El Cano de Xiquena,NO STREET 9999
8,restaurant,40.423345,-3.692632,El Gato Persa,Calle de Doña Bárbara de Braganza 10
9,restaurant,40.422441,-3.694220,Cannibal Raw Bar,Calle del Almirante 12


In [ ]:
pinta_mapa(df)

## Ejercicio 1
### Búsqueda de aparcamiento
Para este caso de uso imagina que vas a Madrid de viaje y estás buscando posibles parkings donde dejar el coche, crea un DataFrame con todos los parkings (de OpenStreetMap) de la ciudad de Madrid. Para ello define una cuadrícula con dos coordenadas sobre Madrid.

Representa en un mapa los resultados

In [ ]:
import requests
import json
import pandas as pd


overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["amenity"="parking"]
  (40.2387,-3.9647,40.5368, -3.2671); 
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()
df = json_to_df(data)
df


#DUDA EL NODO NO SACA LOS PARKINGS DE TODO MADRID. POCOS NUMEROS RESPECTO A LOS DE ALBERTO, ERROR 4OO

<Response [200]>


,tipo,lat,lon,name,address
0,parking,40.394324,-3.794812,NO NAME,NO STREET 9999
1,parking,40.394064,-3.794909,NO NAME,NO STREET 9999
2,parking,40.398227,-3.783833,NO NAME,NO STREET 9999
3,parking,40.426712,-3.714370,Princesa 25,NO STREET 9999
4,parking,40.414469,-3.703549,Parking Benavente,NO STREET 9999
...,...,...,...,...,...
542,parking,40.372956,-3.773751,NO NAME,NO STREET 9999
543,parking,40.395501,-3.794887,NO NAME,NO STREET 9999
544,parking,40.393807,-3.798228,NO NAME,NO STREET 9999
545,parking,40.320327,-3.756167,NO NAME,NO STREET 9999


In [ ]:
data

{'version': 0.6,
 'generator': 'Overpass API 0.7.59 e21c39fe',
 'osm3s': {'timestamp_osm_base': '2022-10-30T17:42:09Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': []}

In [ ]:
mapbox_access_token = "pk.eyJ1IjoiZGFvcnRpIiwiYSI6ImNrZnF6M3FlczA3cDEyem16YTNzZmV4M2EifQ.846iF0sMSAXv0kwkwUTYjg"
def pinta_mapa(df1):
    fig = px.scatter_mapbox(df,
                            lat="lat",
                            lon="lon",
                            color = "tipo",
                      color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=11)
    fig.update_layout(
        mapbox=dict(
            accesstoken=mapbox_access_token
        )
    )
    fig.show()


In [ ]:
pinta_mapa(df1)

## Mudanza
Imagina que te quieres mudar al Berrueco, pero antes te gustaría obtener algún indicador de si hay civilización por esa zona. Mira a ver si existe gasolineras y/o colegios por la zona. Define, al igual que en el caso anterior, un rectángulo mediante dos coordenadas. Busca la manera de realizar una única petición a la API.

Representa después en un mapa, por color los colegios y gasolineras

In [8]:
import json

In [11]:
import requests
import json
import pandas as pd


overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["amenity"~"school|fuel"]
  (40.8637,-3.6071,40.9105,-3.5136); 
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()
df_muda = json_to_df(data)
df_muda

<Response [200]>


,tipo,lat,lon,name,address
0,music_school,40.872876,-3.606421,Escuela Municipal De Musica Y Danza La Cabrera,NO STREET 96
1,fuel,40.875833,-3.602389,Shell,Avenida de La Cabrera 9999


In [15]:
mapbox_access_token = "pk.eyJ1IjoiZGFvcnRpIiwiYSI6ImNrZnF6M3FlczA3cDEyem16YTNzZmV4M2EifQ.846iF0sMSAXv0kwkwUTYjg"
def pinta_mapa(df):
    fig = px.scatter_mapbox(df,
                            lat="lat",
                            lon="lon",
                            color = "tipo",
                      color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=11)
    fig.update_layout(
        mapbox=dict(
            accesstoken=mapbox_access_token
        )
    )
    fig.show()


In [16]:
pinta_mapa(df_muda)

## Zonas COVID
Imagina que trabajas en el ayuntamiento de tu barrio/pueblo/ciudad, y por motivos del COVID tienes que identificar:
1. Todas las zonas abiertas de uso común (zonas de barbacoas, fuentes de agua y parques infantiles (kindergarten)), para cerrarlas.
2. Todos los hospitales y clínicas de la zona.

Como no sabes muy bien qué puntos serán, utilizas la API de OpenStreetMap. Representa todo en un único mapa

In [ ]:
import json
import requests

In [43]:

overpass_url = "https://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["amenity"~"clinic|hospital|kindergarten|drinking_water|bbq"]
  (36.8140, -2.5083,36.8923,-2.3339);
out;
"""
response = requests.get(overpass_url,
                        params={'data': overpass_query})
print(response)
data4 = response.json()
df4 = json_to_df(data4)
df4

<Response [200]>


,tipo,lat,lon,name,address
0,clinic,36.828376,-2.407233,NO NAME,NO STREET 9999
1,drinking_water,36.836433,-2.431740,NO NAME,NO STREET 9999
2,drinking_water,36.828845,-2.405662,NO NAME,NO STREET 9999
3,drinking_water,36.821126,-2.410030,NO NAME,NO STREET 9999
4,drinking_water,36.832898,-2.431008,NO NAME,NO STREET 9999
...,...,...,...,...,...
100,drinking_water,36.882099,-2.435234,NO NAME,NO STREET 9999
101,drinking_water,36.829902,-2.499693,NO NAME,NO STREET 9999
102,drinking_water,36.840319,-2.467041,NO NAME,NO STREET 9999
103,clinic,36.840737,-2.460998,Centro de Salud Almeria Centro (San Leonardo),Calle San Leonardo 7


In [44]:
mapbox_access_token = "pk.eyJ1IjoiZGFvcnRpIiwiYSI6ImNrZnF6M3FlczA3cDEyem16YTNzZmV4M2EifQ.846iF0sMSAXv0kwkwUTYjg"
def pinta_mapa(df):
    fig = px.scatter_mapbox(df,
                            lat="lat",
                            lon="lon",
                            color = "tipo",
                      color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=11)
    fig.update_layout(
        mapbox=dict(
            accesstoken=mapbox_access_token
        )
    )
    fig.show()
pinta_mapa(df4)

## Supermercados
Para este ejemplo supondrás que trabajas en una cadena de supermercados en la que se está planteando construir un nuevo local en la provincia de Sevilla. Busca mediante la API de Overpass todos los supermercados de la provincia para poder comprobar en qué nucleos urbanos habría poca competencia.

En este caso hay que buscar por área, no por cuadrícula! Investiga en la documentación/Stackoverflow cómo se haría eso.

In [35]:
overpass_url = "https://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["shop"="supermarket"]
  (37.2989, -6.1005, 37.4544, -5.7517);
out;
"""
response = requests.get(overpass_url,
                        params={'data': overpass_query})
print(response)
data_super = response.json()
dfsuper = json_to_df(data_super)
dfsuper


<Response [200]>


,tipo,lat,lon,name,address
0,None,37.373002,-6.047204,MAS,NO STREET 9999
1,None,37.390016,-5.973769,Maxi Dia,NO STREET 9999
2,None,37.406364,-5.976342,El Jamón,NO STREET 9999
3,None,37.406508,-5.973342,Valdejimena,Avenida de San Juan de la Salle 9999
4,None,37.406889,-5.972455,Dia,Avenida de San Juan de la Salle 12
...,...,...,...,...,...
234,None,37.393974,-5.992695,Carrefour Express,NO STREET 9999
235,None,37.338372,-5.971192,Tienda Supercor Stop & Go,Avenida de Jerez 24
236,None,37.363353,-6.043652,Family Cash,NO STREET 9999
237,None,37.376837,-5.974045,Dia,Avenida Ramón y Cajal 12


In [31]:
mapbox_access_token = "pk.eyJ1IjoiZGFvcnRpIiwiYSI6ImNrZnF6M3FlczA3cDEyem16YTNzZmV4M2EifQ.846iF0sMSAXv0kwkwUTYjg"
def pinta_mapa(df):
    fig = px.scatter_mapbox(df,
                            lat="lat",
                            lon="lon",
                            color = "tipo",
                      color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=11)
    fig.update_layout(
        mapbox=dict(
            accesstoken=mapbox_access_token
        )
    )
    fig.show()
pinta_mapa(dfsuper)

## Etiqueta en el mapa
Busca la manera de modificar la función `pinta_mapa()`, de tal manera que cuando pases el ratón por encima de cada punto, aparezca el nombre de cada nodo. Por ejemplo, que en el último apartado aparezcan los nombres de los supermercados.

In [ ]:
#PARA SACAR LOS TIPOS DE SUPERMERCADO FILTRADOS EN EL MAPA

dfsuper2 = dfsuper.copy()
dfsuper2["tipo"] = dfsuper["name"]
dfsuper2

In [42]:
def pinta_mapa(df):
    fig = px.scatter_mapbox(df,
                            lat="lat",
                            lon="lon",
                            color = "tipo",
                      color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=11)
    fig.update_layout(
        mapbox=dict(
            accesstoken=mapbox_access_token
        )
    )
    fig.show()
pinta_mapa(dfsuper2)

In [ ]:
overpass_url = "https://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["shop"="supermarket"]
  (37.2989, -6.1005, 37.4544, -5.7517);
out;
"""
response = requests.get(overpass_url,
                        params={'data': overpass_query})
print(response)